In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'NanumGothic'
import sys
import re
import random
import warnings 
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from scipy import optimize
from scipy.stats import levene
from gekko import GEKKO

# Preprocess - 문화향수실태조사 설문조사 데이터

In [4]:
#survey_2014 = pd.read_spss('data/문화향수실태조사_2014.sav')
survey_2016 = pd.read_spss('data/문화향수실태조사_2016.sav')
survey_2018 = pd.read_spss('data/문화향수실태조사_2018.sav')
survey_2019 = pd.read_spss('data/문화향수실태조사_2019.sav')
survey_2019['q22_n8'] = 0

In [5]:
code_2016 = pd.read_excel('data/문화향수실태조사_코드북_2016.xlsx', header=2)
code_2016.fillna(method='ffill', inplace=True)
code_2018 = pd.read_excel('data/문화향수실태조사_코드북_2018.xlsx', header=2)
code_2018.fillna(method='ffill', inplace=True)
code_2019 = pd.read_excel('data/문화향수실태조사_코드북_2019.xlsx', header=0)
code_2019.fillna(method='ffill', inplace=True)

In [6]:
def drop_else_list(year: int):
    
    # match codebook descriptions and dataframe
    df = eval('survey_' + str(year)).copy()
    code = eval('code_' + str(year)).copy()
    drop_list = []
    try: vari = code['변수설명']
    except:
        try: vari = code['설명']
        except: vari = code['변수 설명']
    
    # exclude cultural event which is not performing arts
    for i in range(len(code)):

        if '영화' in str(vari[i]) or '전시회' in str(vari[i]) or '문학행사' in str(vari[i]) or '역사탐방' in str(vari[i]) or '콘서트' in (str(vari[i])):
            if type(code['변수명'][i]) != int:
                if year == 2019:
                    drop_list.append(code['변수명'][i].lower())
                else:
                    drop_list.append(code['변수명'][i])
    
    df.drop(columns=drop_list, inplace=True)
    return df

In [7]:
df_2016 = drop_else_list(2016)
df_2018 = drop_else_list(2018)
df_2019 = drop_else_list(2019)

In [8]:
def scale_convert(x):
    if type(x) == str:
    
        if x == '매우 불만족 한다' or '①' in x: 
            x = 1
        elif x == '불만족 한다' or '②' in x:
            x = 2
        elif x == '다소 불만족 한다' or '③' in x:
            x = 3
        elif x == '보통이다' or '④' in x:
            x = 4
        elif x == '다소 만족한다' or '⑤' in x:
            x = 5
        elif x == '만족한다' or '⑥' in x:
            x = 6
        elif x == '매우 만족한다' or '⑦' in x:
            x = 7
    return x

In [9]:
df_2016 = df_2016.applymap(scale_convert)
df_2018 = df_2018.applymap(scale_convert)
df_2019 = df_2019.applymap(scale_convert)

df_2016['SQ1'] = df_2016['SQ1'].apply(lambda x : x.replace('납', '남'))
df_2018['SQ1'] = df_2018['SQ1'].apply(lambda x : re.sub('[특별라상청자치시도]', '', x).replace('광역', ''))
df_2018['Q22'] = df_2018['Q22'].apply(lambda x : float(x.replace('세', '')))

In [10]:
df_2016.to_csv('data/df_2016.csv')
df_2018.to_csv('data/df_2018.csv')
df_2019.to_csv('data/df_2019.csv')

In [161]:
# how many times have been to certain kind of performance
count_western = {2016:'Q1D1', 2018:'Q1C1', 2019:'q1_1_n3'}
count_tradition = {2016:'Q1E1', 2018:'Q1D1', 2019:'q1_1_n4'}
count_acting = {2016:'Q1F1', 2018:'Q1E1', 2019:'q1_1_n5'}
count_musical = {2016:'Q1G1', 2018:'Q1F1', 2019:'q1_1_n6'}
count_dancing = {2016:'Q1H1', 2018:'Q1G1', 2019:'q1_1_n7'}

# satisfaction score for each performance
satisfy_western = {2016:'Q1D2', 2018:'Q1C2', 2019:'q1_2_3'}
satisfy_tradition = {2016:'Q1E2', 2018:'Q1D2', 2019:'q1_2_4'}
satisfy_acting = {2016:'Q1F2', 2018:'Q1E2', 2019:'q1_2_5'}
satisfy_musical = {2016:'Q1G2', 2018:'Q1F2', 2019:'q1_2_6'}
satisfy_dancing = {2016:'Q1H2', 2018:'Q1G2', 2019:'q1_2_7'}

# demographical variables
marriage = {2016:'Q24', 2018:'Q21', 2019:'q24'}
age = {2016:'Q22', 2018:'Q22', 2019:'q25'}
sex = {2016:'Q21', 2018:'Q25A3', 2019:'q28_3'}
location = {2016:'SQ1', 2018:'SQ1', 2019:'DM11'}
income = {2016:'Q31A2', 2018:'INC', 2019:'DM8'}
edu = {2016:'Q26A1', 2018:'Q25A5', 2019:'q23'}
fam = {2016:'Q20', 2018:'MEMBER', 2019:'q27'}

# 
time = {2016: 'Q3A4', 2018:'Q3A4', 2019:'q3_4'}
supple = {2016: 'Q3A6A1', 2018:'Q3A6A1', 2019:'q3_6'}
standard = {2016: 'Q3A7', 2018: 'Q3A7', 2019:'q3_7'}
prob = {2016: 'Q3A8', 2018: 'Q3A8', 2019:'q3_8'}
companion = {2016:'Q5A4', 2018:'Q5A4', 2019:'q6_4'}

#intend_western = {2016:'Q1D3', 2018:'Q1C3', 2019:'q1_a'}

In [162]:
genres = ['western', 'tradition', 'acting', 'musical', 'dancing']
list_dep = ['count_western', 'count_tradition', 'count_acting', 'count_musical', 'count_dancing',
              'satisfy_western', 'satisfy_tradition', 'satisfy_acting', 'satisfy_musical',
              'satisfy_dancing']
list_cat_indep = ['time', 'supple', 'standard', 'prob', 'companion']
list_indep = ['marriage', 'age', 'sex', 'location', 'income', 'edu', 'fam']

In [163]:
def drop_na(lst):
    return [i for i in lst if str(i) != 'nan']

In [164]:
# concatenate 2016, 2018, 2019 survey data
def combine(lst):
    
    first = True
    for col in lst:
        col_name = col
        col = eval(col)
        df1 = df_2016[col[2016]]
        df2 = df_2018[col[2018]]
        df2 = df2.rename({col[2018]:col[2016]})
        df3 = df_2019[col[2019]]
        df3 = df3.rename({col[2019]:col[2016]})

        #typ = df1[col[2016]].dtype
        #df2[col[2016]] = df2[col[2016]].astype(typ)
        #df3[col[2016]] = df2[col[2016]].astype(typ)

        temp = pd.concat([df1, df2, df3]).reset_index(drop=True)
        if first == True: df = pd.DataFrame(temp, columns=[col_name])
        else: df[col_name] = temp.values
        
        first = False
        
    return df

In [167]:
def age_cat(x):
    try:
        x=int(x)
        if x<10: x='less than 10'
        elif x<20: x='10s'
        elif x<30: x='20s'
        elif x<40: x='30s'
        elif x<50: x='40s'
        elif x<60: x='50s'
        elif x<70: x='60s'
        elif x<80: x='70s'
        else: x='over 80'
        return x
    except:
        return x

In [165]:
lst = list_indep + list_cat_indep + list_dep 
df = combine(lst)

In [168]:
# standardize terms
df = df.applymap(lambda x : x.lstrip() if type(x) == str else x)
df['standard'] = df['standard'].apply(lambda x : x.replace(',네', ', 네').replace(',언', ', 언').replace('(작가, 출연진)', '').replace('즌의', '즌 의').replace('(놀이방 등)', ''))
df['prob'] = df['prob'].apply(lambda x : x.replace('심있', '심 있'))
df['income'] = df['income'].apply(lambda x: re.sub('[월평균만원\s이상미만]', '', x).replace('~', '-'))
df['edu'] = df['edu'].apply(lambda x: re.sub('[박석사과정]', '', str(x).replace(' ', '')))
df['edu'] = df['edu'].apply(lambda x: re.sub('대학원', '대학교(4년제이상)', x))
df['edu'] = df['edu'].apply(lambda x: re.sub('학\(교\)', '학', x))
df['fam'] = df['fam'].apply(lambda x: float(re.sub('[인이상]', '', x)) if type(x)==str else x)
df['companion'] = df['companion'].apply(lambda x: re.sub('친구\(이성친구, 연인\)', '연인', x) if type(x)==str else x)
df['companion'] = df['companion'].apply(lambda x: re.sub('친구\(동성친구\)', '친구', x) if type(x)==str else x)
df['age'] = df['age'].apply(lambda x : age_cat(x))

In [180]:
# make data for regression with genres
tmp1 = df[~df['satisfy_western'].isnull()]
tmp1['genre'] = 'western'
tmp1['satisfaction'] = tmp1['satisfy_western']

tmp2 = df[~df['satisfy_tradition'].isnull()]
tmp2['genre'] = 'tradition'
tmp2['satisfaction'] = tmp1['satisfy_tradition']

tmp3 = df[~df['satisfy_acting'].isnull()]
tmp3['genre'] = 'acting'
tmp3['satisfaction'] = tmp3['satisfy_acting']

tmp4 = df[~df['satisfy_musical'].isnull()]
tmp4['genre'] = 'musical'
tmp4['satisfaction'] = tmp4['satisfy_musical']

tmp5 = df[~df['satisfy_dancing'].isnull()]
tmp5['genre'] = 'dancing'
tmp5['satisfaction'] = tmp5['satisfy_dancing']

tmp6 = pd.concat([tmp1, tmp2, tmp3, tmp4, tmp5])
tmp6.reset_index(inplace=True, drop=True)
tmp6.to_csv('data/data_regression.csv')

In [169]:
#df.to_csv('data/data_preprocessed.csv')

# Preprocess - 인구총조사 데이터

In [6]:
df_pop = pd.read_csv('data/population.csv', encoding='cp949')
df_pop['지역'] = 0
df_pop.drop(columns=['계'], inplace=True)

In [9]:
df_pop['지역'][2:54] = '서울'
df_pop['지역'][54:88] = '부산'
df_pop['지역'][88:106] = '대구'
df_pop['지역'][106:128] = '인천'
df_pop['지역'][128:140] = '광주'
df_pop['지역'][140:152] = '대전'
df_pop['지역'][152:164] = '울산'
df_pop['지역'][164:168] = '세종'
df_pop['지역'][168:266] = '경기'
df_pop['지역'][266:304] = '강원'
df_pop['지역'][304:336] = '충북'
df_pop['지역'][336:372] = '충남'
df_pop['지역'][372:406] = '전북'
df_pop['지역'][406:452] = '전남'
df_pop['지역'][452:504] = '경북'
df_pop['지역'][504:552] = '경남'
df_pop['지역'][552:] = '제주'

In [10]:
def concat_col(df, n):
    return df.iloc[:, n] + df.iloc[:, n+1]

In [11]:
# change columns from 5 year standard to 10 year standard
total = 0
for i in (range(3, 20, 2)):
    df_pop.iloc[:, i] = (concat_col(df_pop, i))

drop_col = list(df_pop.columns[4:20:2]) + list(df_pop.columns[20:-1])
df_pop.drop(columns=drop_col, inplace=True)

In [13]:
df_pop.columns = ['시점', '시군구', 'sex', 
                  'less than 10', '10s', '20s', '30s', '40s', '50s', '60s', '70s', 'over 80',
                  '지역']

In [15]:
#df_pop.to_csv('data/data_population.csv')

# Preprocess - 공연 DB, 공연장 DB